In [ ]:
import gzip
import os
import sys
import urllib
import matplotlib.image as mpimg
from PIL import Image

import code
import os
#import tensorflow.python.platform

import numpy as np
import tensorflow.keras as keras
from keras import layers, models, datasets,regularizers, Model, Input
from keras.layers import concatenate, Conv2D, Conv2DTranspose, BatchNormalization, MaxPooling2D, Dropout
from helpers_images import * 


In [ ]:
if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

GPU found


In [ ]:
RATIO_VALIDATION = 0.1
NUMBER_TRAINING_EXAMPLES = 100
NUMBER_TEST_EXAMPLES = 50
SAVE_MODEL = False
LOAD_MODEL = False
DATA_LOADED = False

In [ ]:
training_data_directory = "../data/training/"
test_data_directory = "../data/test_set_images/"
if not(DATA_LOADED):    
    train_data_orig = extract_train_data(NUMBER_TRAINING_EXAMPLES)
    train_labels_orig =  extract_labels(training_data_directory, NUMBER_TRAINING_EXAMPLES,True)
    DATA_LOADED = True

print(train_data_orig.shape)
print(train_labels_orig.shape)

Loading ./drive/MyDrive/Projet2ML/data/training/images/satImage_001.png
Loading ./drive/MyDrive/Projet2ML/data/training/images/satImage_002.png
Loading ./drive/MyDrive/Projet2ML/data/training/images/satImage_003.png
Loading ./drive/MyDrive/Projet2ML/data/training/images/satImage_004.png
Loading ./drive/MyDrive/Projet2ML/data/training/images/satImage_005.png
Loading ./drive/MyDrive/Projet2ML/data/training/images/satImage_006.png
Loading ./drive/MyDrive/Projet2ML/data/training/images/satImage_007.png
Loading ./drive/MyDrive/Projet2ML/data/training/images/satImage_008.png
Loading ./drive/MyDrive/Projet2ML/data/training/images/satImage_009.png
Loading ./drive/MyDrive/Projet2ML/data/training/images/satImage_010.png
Loading ./drive/MyDrive/Projet2ML/data/training/images/satImage_011.png
Loading ./drive/MyDrive/Projet2ML/data/training/images/satImage_012.png
Loading ./drive/MyDrive/Projet2ML/data/training/images/satImage_013.png
Loading ./drive/MyDrive/Projet2ML/data/training/images/satImage_

In [ ]:
 IMG_PATCH_SIZE = 400
train_data = train_data_orig
train_labels = train_labels_orig

In [ ]:
def encoding_block(input, nbr_filters = 16, pooling_ = True, dropout = True):
    conv = Conv2D(nbr_filters, (3, 3), activation='elu',kernel_regularizer=regularizers.l2(0.0001), kernel_initializer='he_normal', padding='same') (input)
    conv = BatchNormalization()(conv)
    if dropout:
      conv = Dropout(0.1) (conv)
    conv = Conv2D(nbr_filters, (3, 3), activation='elu',kernel_regularizer=regularizers.l2(0.0001), kernel_initializer='he_normal', padding='same') (conv)
    conv = BatchNormalization()(conv)
    if pooling_:
      pooling = MaxPooling2D((2,2))(conv)
      return conv, pooling
    return conv

In [ ]:
def decoding_block(conv, previous_conv,nbr_filters = 16, dropout = True):
  upsample = Conv2DTranspose(nbr_filters, (2, 2),kernel_regularizer=regularizers.l2(0.01), strides=(2, 2), padding='same') (conv)
  upsample = concatenate([upsample, previous_conv])
  conv = Conv2D(nbr_filters, (3, 3), activation='elu',kernel_regularizer=regularizers.l2(0.0001), kernel_initializer='he_normal', padding='same') (upsample)
  conv = BatchNormalization() (conv)
  if dropout:
    conv = Dropout(0.2) (conv)
  conv = Conv2D(nbr_filters, (3, 3), activation='elu',kernel_regularizer=regularizers.l2(0.0001), kernel_initializer='he_normal', padding='same') (conv)
  conv = BatchNormalization() (conv)
  return conv

In [ ]:
def create_custom_unet(nbr_filters = 16, dropout = True):
  inputs = Input((IMG_PATCH_SIZE, IMG_PATCH_SIZE, 3))
  #augmented = layers.RandomContrast(0.2)(inputs)
  conv1, pooling1 = encoding_block(inputs, nbr_filters, True, dropout)
  conv2, pooling2 = encoding_block(pooling1, nbr_filters*2, True,dropout)
  conv3, pooling3 = encoding_block(pooling2, nbr_filters*4, True,dropout)
  conv4, pooling4 = encoding_block(pooling3, nbr_filters*8, True,dropout)
  conv5 = encoding_block(pooling4, nbr_filters*16,False, dropout)

  conv6 = decoding_block(conv5, conv4, nbr_filters*8, dropout)
  conv7 = decoding_block(conv6, conv3, nbr_filters*4, dropout)
  conv8 = decoding_block(conv7, conv2, nbr_filters*2, dropout)
  conv9 = decoding_block(conv8, conv1, nbr_filters, dropout)
  outputs = Conv2D(1, (1, 1), activation='sigmoid') (conv9)

  model = Model(inputs=[inputs], outputs=[outputs])
  return model


In [ ]:
from keras import backend as K

def iou_coef(y_true, y_pred, smooth=1):
    intersection = K.sum(K.abs(y_true * y_pred), axis=[1,2,3])
    union = K.sum(y_true,[1,2,3])+K.sum(y_pred,[1,2,3])-intersection
    iou = K.mean((intersection + smooth) / (union + smooth), axis=0)

    return iou

def dice_coef(y_true, y_pred, smooth = 1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def soft_dice_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
earlystopper = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0, 
                          patience = 5,
                          verbose = 1,
                          mode = "min",
                          restore_best_weights = True)

lr_reducer = ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=4,
                               verbose=1,
                               mode="min",
                               min_delta=1e-4)

In [ ]:
model = create_custom_unet(16, True)
model.summary()
model.compile(optimizer='adam',
            loss=soft_dice_loss,
          metrics=[dice_coef])y       

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 400, 400, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 400, 400, 16  448         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 400, 400, 16  64         ['conv2d[0][0]']                 
 alization)                     )                                                             

'model.compile(optimizer=\'adam\',\n            loss=\'binary_crossentropy\',\n          metrics=["Precision", "Recall"])'

In [ ]:
epochs = 200
history = model.fit(train_data, train_labels, epochs=epochs, validation_split = 0.01, batch_size = 8,callbacks = [ earlystopper, lr_reducer])

Epoch 1/200
112/112 [==============================] - 77s 543ms/step - loss: 1.2776 - dice_coef: 0.5828 - val_loss: 0.5387 - val_dice_coef: 0.6272 - lr: 0.0010
Epoch 2/200
112/112 [==============================] - 57s 506ms/step - loss: 0.4019 - dice_coef: 0.7456 - val_loss: 0.4259 - val_dice_coef: 0.6610 - lr: 0.0010
Epoch 3/200
112/112 [==============================] - 57s 505ms/step - loss: 0.3286 - dice_coef: 0.7901 - val_loss: 0.3561 - val_dice_coef: 0.7177 - lr: 0.0010
Epoch 4/200
112/112 [==============================] - 57s 505ms/step - loss: 0.2939 - dice_coef: 0.8146 - val_loss: 0.2734 - val_dice_coef: 0.7916 - lr: 0.0010
Epoch 5/200
112/112 [==============================] - 56s 504ms/step - loss: 0.2718 - dice_coef: 0.8298 - val_loss: 0.2752 - val_dice_coef: 0.8007 - lr: 0.0010
Epoch 6/200
112/112 [==============================] - 57s 506ms/step - loss: 0.2559 - dice_coef: 0.8407 - val_loss: 0.2629 - val_dice_coef: 0.7855 - lr: 0.0010
Epoch 7/200
112/112 [=============

In [ ]:
import pathlib
directory_path = pathlib.Path("./drive/MyDrive/Projet2ML/data/test_set_images/")

In [ ]:
all_images = []
for i in range(1,51):
    image_in_directory = list(directory_path.glob("test_" + str(i) + "/*"))
    all_images.extend(image_in_directory)
img_height = img_width = 608

In [ ]:
all_images_arrays = []
for i in range(len(all_images)):
    img = tf.keras.utils.load_img(
        all_images[i], target_size=(img_height, img_width)
    )
    img_array = tf.keras.utils.img_to_array(img)
    all_images_arrays.append(img_array)
all_images_arrays = numpy.array(all_images_arrays)/255.0

In [ ]:
"""test_images = []
for i in range(len(all_images_arrays)):
    img_patched = extract_data_from_image(all_images_arrays[i])
    test_images.extend(img_patched)
test_images = numpy.array(test_images)"""

'test_images = []\nfor i in range(len(all_images_arrays)):\n    img_patched = extract_data_from_image(all_images_arrays[i])\n    test_images.extend(img_patched)\ntest_images = numpy.array(test_images)'

In [ ]:
predictions_testing = "./drive/MyDrive/Projet2ML/testing/predictions_testing/"
if not os.path.isdir(predictions_testing):
    os.mkdir(predictions_testing)

In [ ]:
def predict(k):
    img = all_images_arrays[k]
    extract_1 = img[:400, :400]
    extract_2 = img[:400, -400:]
    extract_3 = img[-400:, :400]
    extract_4 = img[-400:, -400:]
    output_prediction = []
    output_prediction.append(model(np.expand_dims(extract_1 , axis=0)).numpy() )
    output_prediction.append(model(np.expand_dims(extract_2 , axis=0)).numpy() )
    output_prediction.append(model(np.expand_dims(extract_3 , axis=0)).numpy() )
    output_prediction.append(model(np.expand_dims(extract_4 , axis=0)).numpy() )

    output_prediction[0] = output_prediction[0].reshape(400,400)
    output_prediction[1] = output_prediction[1].reshape(400,400)
    output_prediction[2] = output_prediction[2].reshape(400,400)
    output_prediction[3] = output_prediction[3].reshape(400,400)

    #output_prediction[output_prediction > 0.5] = 1
    #output_prediction[output_prediction <= 0.5] = 0
    gt_img[:400,:400] = output_prediction[0]
    gt_img[:400, -400:] = output_prediction[1]
    gt_img[-400: , :400] = output_prediction[2]
    gt_img[-400: , -400:] = output_prediction[3]
    """idx = 0
    for i in range(0, img.shape[0], IMG_PATCH_SIZE):
        for j in range(0, img.shape[1], IMG_PATCH_SIZE):
            pred = output_prediction[idx]
            gt_img[j:j + IMG_PATCH_SIZE, i:i+IMG_PATCH_SIZE] = pred.reshape(IMG_PATCH_SIZE,IMG_PATCH_SIZE)
            idx += 1"""
    oimg = make_img_overlay(img, gt_img)
    gt_img8 = img_float_to_uint8(gt_img)          
    gt_img_3c = numpy.zeros((img.shape[0], img.shape[1], 3), dtype=numpy.uint8)
    gt_img_3c[:, :, 0] = gt_img8
    gt_img_3c[:, :, 1] = gt_img8
    gt_img_3c[:, :, 2] = gt_img8
    overlay = make_img_overlay(img, gt_img)
    Image.fromarray(gt_img_3c).save(predictions_testing + "prediction_image_" + str(k+1) + ".png")
    oimg.save(predictions_testing  + "overlay_" + str(k) + ".png")

In [ ]:
for i in range(len(all_images_arrays)):
    predict(i)

TypeError: ignored

In [ ]:
5

5

In [ ]:
#model.save("./drive/MyDrive/Projet2ML/testing/model_0.876.h5")